# Description
Immunologic clock based on combined data (Сentral region + Yakutia).
Previously, the clock was built only on data from the central region, here the clock is built on data from both regions.
For simplicity, only the best model type at the time of script creation - FT-Transformer - is trained.

In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from scipy import stats
import plotly.express as px
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import seaborn as sns
from glob import glob
import pathlib
from sklearn.metrics import mean_absolute_error
from scipy import stats
import patchworklib as pw
import os
import functools
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import shap
from slugify import slugify
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from art.estimators.regression.pytorch import PyTorchRegressor
from art.estimators.regression.blackbox import BlackBoxRegressor
from art.attacks.evasion import LowProFool, ZooAttack, FastGradientMethod
import torch
from sklearn.model_selection import RepeatedStratifiedKFold


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/049_immuno_age_regression_extended"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

In [ ]:
df_all = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
ids_yakutia = df_all.index[df_all["Region"] == "Yakutia"].values
tmp = pd.read_excel(f"{path}/special/044_small_immuno_clocks_revision/figure_simage/df.xlsx", index_col=0)
ids_central = tmp.index[tmp["Status"] == "Control"].values

In [ ]:
df_res = df_all.loc[np.concatenate([ids_central, ids_yakutia]), :]

geo_parts = {
    'Cental': df_all.loc[ids_central, ['Age']].copy(),
    'Yakutia': df_all.loc[ids_yakutia, ['Age']].copy(),
}
trn_val_ids = {}
tst_ids = {}
for part, df_part in geo_parts.items():
    trgt = df_part.loc[:, "Age"].values
    ids = df_part.index.values

    ptp = np.ptp(trgt)
    num_bins = 5
    bins = np.linspace(np.min(trgt) - 0.1 * ptp, np.max(trgt) + 0.1 * ptp, num_bins + 1)
    binned = np.digitize(trgt, bins) - 1
    unique, counts = np.unique(binned, return_counts=True)
    occ = dict(zip(unique, counts))

    k_fold = RepeatedStratifiedKFold(
        n_splits=5,
        n_repeats=1,
        random_state=1337
    )
    splits = k_fold.split(X=ids, y=binned, groups=binned)

    for split_id, (ids_trn, ids_val) in enumerate(splits):
        df_res.loc[ids[ids_trn], f"Split_{split_id}"] = "trn_val"
        df_res.loc[ids[ids_val], f"Split_{split_id}"] = "tst"

df_res.to_excel(f"{path_save}/data.xlsx", index_label="index")

hist_bins = np.linspace(0, 110, 12)

palette = {'trn_val': 'deepskyblue', 'tst': 'crimson'}
df_fig = df_res.loc[ids_central, :]
fig = plt.figure()
sns.set_theme(style='ticks', font_scale=1.3)
hist = sns.histplot(
    data=df_fig,
    hue_order=['trn_val', 'tst'],
    bins=hist_bins,
    x="Age",
    hue="Split_0",
    edgecolor='black',
    palette=palette,
    multiple="stack",
)
hist.set(xlim=(0, 110))
sns.despine(left=False, right=True, bottom=False, top=True)
plt.savefig(f"{path_save}/hist_central.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_save}/hist_central.pdf", bbox_inches='tight')
plt.close(fig)

palette = {'trn_val': 'blue', 'tst': 'darkred'}
df_fig = df_res.loc[ids_yakutia, :]
fig = plt.figure()
sns.set_theme(style='ticks', font_scale=1.3)
hist = sns.histplot(
    data=df_fig,
    hue_order=['trn_val', 'tst'],
    bins=hist_bins,
    x="Age",
    hue="Split_0",
    edgecolor='black',
    palette=palette,
    multiple="stack",
)
hist.set(xlim=(0, 110))
sns.despine(left=False, right=True, bottom=False, top=True)
plt.savefig(f"{path_save}/hist_yakutia.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_save}/hist_yakutia.pdf", bbox_inches='tight')
plt.close(fig)

In [ ]:
model = 'widedeep_ft_transformer_trn_val_tst'

path_runs = f"{path_save}/models/{model}/multiruns"

files = glob(f"{path_runs}/*/*/metrics_all_best_*.xlsx")

df_tmp = pd.read_excel(files[0], index_col="metric")
head, tail = os.path.split(files[0])
cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
params = []
for param_pair in cfg:
    param, val = param_pair.split('=')
    params.append(param)
df_res = pd.DataFrame(index=files)

for file in files:
    # Metrics
    df_metrics = pd.read_excel(file, index_col="metric")
    for metric in df_metrics.index.values:
        df_res.at[file, metric + "_val"] = df_metrics.at[metric, "val"]
        df_res.at[file, metric + "_trn"] = df_metrics.at[metric, "trn"]
        df_res.at[file, metric + "_tst"] = df_metrics.at[metric, "tst"]
        df_res.at[file, metric + "_trn_val"] = df_metrics.at[metric, "trn_val"]
        df_res.at[file, metric + "_val_tst"] = df_metrics.at[metric, "val_tst"]
        df_res.at[file, metric + "_trn_val_tst"] = df_metrics.at[metric, "trn_val_tst"]

    # Params
    cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
    for param_pair in cfg:
        param, val = param_pair.split('=')
        df_res.at[file, param] = val

df_res["train_more_val"] = False
df_res["selected"] = False
df_res.loc[df_res["mean_absolute_error_trn"] > df_res["mean_absolute_error_val"], "train_more_val"] = True

first_columns = [
    'selected',
    'train_more_val',
    'mean_absolute_error_trn',
    'mean_absolute_error_val',
    'mean_absolute_error_tst',
    'mean_absolute_error_val_tst',
    'mean_absolute_error_trn_val_tst',
    'pearson_corr_coef_trn',
    'pearson_corr_coef_val',
    'pearson_corr_coef_tst',
    'pearson_corr_coef_val_tst',
    'pearson_corr_coef_trn_val_tst',
    'mean_absolute_error_cv_mean_trn',
    'mean_absolute_error_cv_std_trn',
    'mean_absolute_error_cv_mean_val',
    'mean_absolute_error_cv_std_val',
    'pearson_corr_coef_cv_mean_trn',
    'pearson_corr_coef_cv_std_trn',
    'pearson_corr_coef_cv_mean_val',
    'pearson_corr_coef_cv_std_val',
]
df_res = df_res[first_columns + [col for col in df_res.columns if col not in first_columns]]
df_res.to_excel(f"{path_runs}/summary.xlsx", index=True, index_label="file")